In [2]:
from text_parse import *

code = """
my_list = [1, 2, 3, 4, 5]
min = my_list[0]
for i in my_list:
    if i < min:
        min = i
print(min)
"""
        
print(tokenize(code))


[['word', 'my_list'], ['symbol', '=>'], ['symbol', '['], ['number', '1'], ['symbol', ','], ['number', '2'], ['symbol', ','], ['number', '3'], ['symbol', ','], ['number', '4'], ['symbol', ','], ['number', '5'], ['symbol', ']'], ['word', 'min'], ['symbol', '='], ['word', 'my_list'], ['symbol', '['], ['number', '0'], ['symbol', ']'], ['word', 'for'], ['word', 'i'], ['word', 'in'], ['word', 'my_list'], ['symbol', ':'], ['word', 'if'], ['word', 'i'], ['symbol', '<'], ['word', 'min'], ['symbol', ':'], ['word', 'min'], ['symbol', '='], ['word', 'i'], ['word', 'print'], ['symbol', '('], ['word', 'min'], ['symbol', ')']]
